# 第九章：从零开始构建复杂提示


## 初始设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:
# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

---

## 课程内容 {#lesson内容}

恭喜您来到最后一章！现在是时候将所有内容整合起来，学习如何**创建独特而复杂的提示**。

接下来，您将使用我们为复杂提示推荐的**指导性结构**。在本章的后续部分，我们将向您展示一些行业特定的提示，并解释这些提示如何采用类似的结构。

**注意：** **并非所有提示都需要以下复杂结构的每个元素**。我们鼓励您尝试包含或排除某些元素，看看这如何影响GPT的响应。通常**最好先使用多个提示元素让您的提示正常工作，然后再优化和精简您的提示**。

### 示例 - 职业教练聊天机器人

以下结构结合了多个提示工程元素，是复杂提示的良好起点。**对某些元素来说顺序很重要**，对其他元素则不然。我们会注明最佳实践何时指示顺序重要，但总的来说，**如果您坚持这个顺序，这将是一个卓越提示的良好开端**。

在下面的示例中，我们将构建一个受控角色扮演的提示，其中GPT承担具有特定任务的情境角色。我们的目标是提示GPT扮演一个友好的职业教练。

阅读然后运行下面的单元格，将各种提示元素编译成一个完整的提示。

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 对话历史（这也可以作为API调用中的前置`user`和`assistant`消息添加）
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# 第二个输入变量 - 用户的问题
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## 提示元素 ########################################

##### 提示元素 1: `user` 角色
# 确保您的Messages API调用始终以消息数组中的`user`角色开始。
# 上面定义的get_completion()函数将自动为您执行此操作。

##### 提示元素 2: 任务上下文
# 给GPT提供关于它应该承担的角色或您希望它通过提示承担的目标和总体任务的上下文。
# 最好将上下文放在提示主体的早期。
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### 提示元素 3: 语调上下文
# 如果对交互很重要，告诉GPT应该使用什么语调。
# 根据任务，此元素可能不是必需的。
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### 提示元素 4: 详细任务描述和规则
# 详细说明您希望GPT执行的具体任务，以及GPT可能必须遵循的任何规则。
# 这也是您可以给GPT一个"出路"的地方，如果它没有答案或不知道。
# 理想情况下，向朋友展示此描述和规则，确保逻辑清晰，任何模糊的词语都被明确定义。
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### 提示元素 5: 示例
# 为GPT提供至少一个它可以模仿的理想响应示例。将其封装在<example></example> XML标签中。随意提供多个示例。
# 如果您确实提供多个示例，请给GPT提供关于这是什么示例的上下文，并将每个示例封装在其自己的XML标签集中。
# 示例可能是知识工作中让GPT按需要行为的最有效的单一工具。
# 确保给GPT提供常见边缘情况的示例。如果您的提示使用草稿本，提供草稿本应该如何看起来的示例是有效的。
# 通常更多示例 = 更好。
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### 提示元素 6: 要处理的输入数据
# 如果GPT需要在提示中处理数据，请在相关XML标签中包含它。
# 随意包含多个数据片段，但确保将每个都封装在其自己的XML标签集中。
# 根据任务，此元素可能不是必需的。顺序也是灵活的。
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### 提示元素 7: 即时任务描述或请求 #####
# "提醒"GPT或告诉GPT它期望立即做什么来完成提示的任务。
# 这也是您放置用户问题等附加变量的地方。
# 重申GPT的即时任务通常没有坏处。最好在长提示的末尾这样做。
# 这将产生比在开始时放置此内容更好的结果。
# 将用户的查询放在提示底部附近通常也是良好的实践。
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### 提示元素 8: 预知（逐步思考）
# 对于多步骤任务，最好告诉GPT在给出答案之前逐步思考
# 有时，您甚至可能必须说"在您给出答案之前..."只是为了确保GPT首先这样做。
# 并非所有提示都必需，但如果包含，最好在长提示的末尾和最终即时任务请求或描述之后执行此操作。
PRECOGNITION = "Think about your answer first before you respond."

##### 提示元素 9: 输出格式化
# 如果您希望GPT的响应有特定格式，请清楚地告诉GPT该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果您包含它，将其放在提示末尾比在开始时更好。
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### 提示元素 10: 预填充GPT的响应（如果有）
# 一个空间，用一些预填充的词语开始GPT的答案，以引导GPT的行为或响应。
# 如果您想预填充GPT的响应，您必须将其放在API调用中的`assistant`角色中。
# 根据任务，此元素可能不是必需的。
PREFILL = "[Joe] <response>"



######################################## 组合元素 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)

现在让我们运行提示！运行下面的单元格来查看GPT的输出。

In [ ]:
print("\n------------------------------------- GPT的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

### 示例 - 法律服务

**法律专业内的提示可能相当复杂**，因为需要：
- 解析长文档
- 处理复杂主题
- 以非常特定的方式格式化输出
- 遵循多步骤分析过程

让我们看看如何使用复杂提示模板来构建特定法律用例的提示。下面，我们详细说明了一个法律用例的示例提示，其中我们要求GPT使用法律文档中的信息回答有关法律问题的问题。

我们**改变了一些元素的顺序**来展示提示结构可以是灵活的！

**提示工程是关于科学的试验和错误**。我们鼓励您混合搭配，移动元素（在顺序不重要的元素），看看什么最适合您和您的需求。

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the legal document
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Second input variable - the user's question
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give GPT context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Prompt element 3: Tone context
# If important to the interaction, tell GPT what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that GPT needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Prompt element 5: Examples
# Provide GPT with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give GPT context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting GPT to behave as desired.
# Make sure to give GPT examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want GPT to do, as well as any rules that GPT might have to follow.
# This is also where you can give GPT an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Prompt element 7: Immediate task description or request #####
# "Remind" GPT or tell GPT exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to GPT its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell GPT to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure GPT does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Prompt element 9: Output formatting
# If there is a specific way you want GPT's response formatted, clearly tell GPT what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Prompt element 10: Prefilling GPT's response (if any)
# A space to start off GPT's answer with some prefilled words to steer GPT's behavior or response.
# If you want to prefill GPT's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

Now let's run the prompt! Run the cell below to see GPT's output.

In [ ]:
print("\n------------------------------------- GPT's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

### Exercise 9.1 - Financial Services Chatbot
Prompts within the financial profession can also be quite complex due to reasons similar to legal prompts. Here's an exercise for a financial use-case, wherein GPT is used to **analyze tax information and answer questions**. Just like with the legal services example, we've changed around the ordering of a few elements, as our solution prompt makes more sense with a different flow (however, other structures would also work).

We suggest you read through the variable content (in this case, `{QUESTION}` and `{TAX_CODE}`) to understand what content GPT is expected to work with. Be sure to reference `{QUESTION}` and `{TAX_CODE}` directly in your prompt somewhere (using f-string syntax like in the other examples) so that the actual variable content can be substituted in.

Fill in the prompt element fields with content that match the description and the examples you've seen in the preceding examples of complex prompts. Once you have filled out all the prompt elements that you want to fill out, run the cell to see the concatenated prompt as well as GPT's response.

Remember that prompt engineering is rarely purely formulaic, especially for large and complex prompts! It's important to develop test cases and **try a variety of prompts and prompt structures to see what works best for each situation**. Note that if you *do* change the ordering of the prompt elements, you should also remember to change the ordering of the concatenaton in the `COMBINE ELEMENTS` section.

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the user's question
QUESTION = "How long do I have to make an 83b election?"

# Second input variable - the tax code document that Claude will be using to answer the user's question
TAX_CODE = """
(a)General rule
If, in connection with the performance of services, property is transferred to any person other than the person for whom such services are performed, the excess of—
(1)the fair market value of such property (determined without regard to any restriction other than a restriction which by its terms will never lapse) at the first time the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, over
(2)the amount (if any) paid for such property,
shall be included in the gross income of the person who performed such services in the first taxable year in which the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever is applicable. The preceding sentence shall not apply if such person sells or otherwise disposes of such property in an arm’s length transaction before his rights in such property become transferable or not subject to a substantial risk of forfeiture.
(b)Election to include in gross income in year of transfer
(1)In general
Any person who performs services in connection with which property is transferred to any person may elect to include in his gross income for the taxable year in which such property is transferred, the excess of—
(A)the fair market value of such property at the time of transfer (determined without regard to any restriction other than a restriction which by its terms will never lapse), over
(B)the amount (if any) paid for such property.
If such election is made, subsection (a) shall not apply with respect to the transfer of such property, and if such property is subsequently forfeited, no deduction shall be allowed in respect of such forfeiture.
(2)Election
An election under paragraph (1) with respect to any transfer of property shall be made in such manner as the Secretary prescribes and shall be made not later than 30 days after the date of such transfer. Such election may not be revoked except with the consent of the Secretary.

(c)Special rules
For purposes of this section—
(1)Substantial risk of forfeiture
The rights of a person in property are subject to a substantial risk of forfeiture if such person’s rights to full enjoyment of such property are conditioned upon the future performance of substantial services by any individual.

(2)Transferability of property
The rights of a person in property are transferable only if the rights in such property of any transferee are not subject to a substantial risk of forfeiture.

(3)Sales which may give rise to suit under section 16(b) of the Securities Exchange Act of 1934
So long as the sale of property at a profit could subject a person to suit under section 16(b) of the Securities Exchange Act of 1934, such person’s rights in such property are—
(A)subject to a substantial risk of forfeiture, and
(B)not transferable.
(4)For purposes of determining an individual’s basis in property transferred in connection with the performance of services, rules similar to the rules of section 72(w) shall apply.
(d)Certain restrictions which will never lapse
(1)Valuation
In the case of property subject to a restriction which by its terms will never lapse, and which allows the transferee to sell such property only at a price determined under a formula, the price so determined shall be deemed to be the fair market value of the property unless established to the contrary by the Secretary, and the burden of proof shall be on the Secretary with respect to such value.

(2)Cancellation
If, in the case of property subject to a restriction which by its terms will never lapse, the restriction is canceled, then, unless the taxpayer establishes—
(A)that such cancellation was not compensatory, and
(B)that the person, if any, who would be allowed a deduction if the cancellation were treated as compensatory, will treat the transaction as not compensatory, as evidenced in such manner as the Secretary shall prescribe by regulations,
the excess of the fair market value of the property (computed without regard to the restrictions) at the time of cancellation over the sum of—
(C)the fair market value of such property (computed by taking the restriction into account) immediately before the cancellation, and
(D)the amount, if any, paid for the cancellation,
shall be treated as compensation for the taxable year in which such cancellation occurs.
(e)Applicability of section
This section shall not apply to—
(1)a transaction to which section 421 applies,
(2)a transfer to or from a trust described in section 401(a) or a transfer under an annuity plan which meets the requirements of section 404(a)(2),
(3)the transfer of an option without a readily ascertainable fair market value,
(4)the transfer of property pursuant to the exercise of an option with a readily ascertainable fair market value at the date of grant, or
(5)group-term life insurance to which section 79 applies.
(f)Holding period
In determining the period for which the taxpayer has held property to which subsection (a) applies, there shall be included only the period beginning at the first time his rights in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier.

(g)Certain exchanges
If property to which subsection (a) applies is exchanged for property subject to restrictions and conditions substantially similar to those to which the property given in such exchange was subject, and if section 354, 355, 356, or 1036 (or so much of section 1031 as relates to section 1036) applied to such exchange, or if such exchange was pursuant to the exercise of a conversion privilege—
(1)such exchange shall be disregarded for purposes of subsection (a), and
(2)the property received shall be treated as property to which subsection (a) applies.
(h)Deduction by employer
In the case of a transfer of property to which this section applies or a cancellation of a restriction described in subsection (d), there shall be allowed as a deduction under section 162, to the person for whom were performed the services in connection with which such property was transferred, an amount equal to the amount included under subsection (a), (b), or (d)(2) in the gross income of the person who performed such services. Such deduction shall be allowed for the taxable year of such person in which or with which ends the taxable year in which such amount is included in the gross income of the person who performed such services.

(i)Qualified equity grants
(1)In general
For purposes of this subtitle—
(A)Timing of inclusion
If qualified stock is transferred to a qualified employee who makes an election with respect to such stock under this subsection, subsection (a) shall be applied by including the amount determined under such subsection with respect to such stock in income of the employee in the taxable year determined under subparagraph (B) in lieu of the taxable year described in subsection (a).

(B)Taxable year determined
The taxable year determined under this subparagraph is the taxable year of the employee which includes the earliest of—
(i)the first date such qualified stock becomes transferable (including, solely for purposes of this clause, becoming transferable to the employer),
(ii)the date the employee first becomes an excluded employee,
(iii)the first date on which any stock of the corporation which issued the qualified stock becomes readily tradable on an established securities market (as determined by the Secretary, but not including any market unless such market is recognized as an established securities market by the Secretary for purposes of a provision of this title other than this subsection),
(iv)the date that is 5 years after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, or
(v)the date on which the employee revokes (at such time and in such manner as the Secretary provides) the election under this subsection with respect to such stock.
(2)Qualified stock
(A)In general
For purposes of this subsection, the term “qualified stock” means, with respect to any qualified employee, any stock in a corporation which is the employer of such employee, if—
(i)such stock is received—
(I)in connection with the exercise of an option, or
(II)in settlement of a restricted stock unit, and
(ii)such option or restricted stock unit was granted by the corporation—
(I)in connection with the performance of services as an employee, and
(II)during a calendar year in which such corporation was an eligible corporation.
(B)Limitation
The term “qualified stock” shall not include any stock if the employee may sell such stock to, or otherwise receive cash in lieu of stock from, the corporation at the time that the rights of the employee in such stock first become transferable or not subject to a substantial risk of forfeiture.

(C)Eligible corporation
For purposes of subparagraph (A)(ii)(II)—
(i)In general
The term “eligible corporation” means, with respect to any calendar year, any corporation if—
(I)no stock of such corporation (or any predecessor of such corporation) is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) during any preceding calendar year, and
(II)such corporation has a written plan under which, in such calendar year, not less than 80 percent of all employees who provide services to such corporation in the United States (or any possession of the United States) are granted stock options, or are granted restricted stock units, with the same rights and privileges to receive qualified stock.
(ii)Same rights and privileges
For purposes of clause (i)(II)—
(I)except as provided in subclauses (II) and (III), the determination of rights and privileges with respect to stock shall be made in a similar manner as under section 423(b)(5),
(II)employees shall not fail to be treated as having the same rights and privileges to receive qualified stock solely because the number of shares available to all employees is not equal in amount, so long as the number of shares available to each employee is more than a de minimis amount, and
(III)rights and privileges with respect to the exercise of an option shall not be treated as the same as rights and privileges with respect to the settlement of a restricted stock unit.
(iii)Employee
For purposes of clause (i)(II), the term “employee” shall not include any employee described in section 4980E(d)(4) or any excluded employee.

(iv)Special rule for calendar years before 2018
In the case of any calendar year beginning before January 1, 2018, clause (i)(II) shall be applied without regard to whether the rights and privileges with respect to the qualified stock are the same.

(3)Qualified employee; excluded employee
For purposes of this subsection—
(A)In general
The term “qualified employee” means any individual who—
(i)is not an excluded employee, and
(ii)agrees in the election made under this subsection to meet such requirements as are determined by the Secretary to be necessary to ensure that the withholding requirements of the corporation under chapter 24 with respect to the qualified stock are met.
(B)Excluded employee
The term “excluded employee” means, with respect to any corporation, any individual—
(i)who is a 1-percent owner (within the meaning of section 416(i)(1)(B)(ii)) at any time during the calendar year or who was such a 1 percent owner at any time during the 10 preceding calendar years,
(ii)who is or has been at any prior time—
(I)the chief executive officer of such corporation or an individual acting in such a capacity, or
(II)the chief financial officer of such corporation or an individual acting in such a capacity,
(iii)who bears a relationship described in section 318(a)(1) to any individual described in subclause (I) or (II) of clause (ii), or
(iv)who is one of the 4 highest compensated officers of such corporation for the taxable year, or was one of the 4 highest compensated officers of such corporation for any of the 10 preceding taxable years, determined with respect to each such taxable year on the basis of the shareholder disclosure rules for compensation under the Securities Exchange Act of 1934 (as if such rules applied to such corporation).
(4)Election
(A)Time for making election
An election with respect to qualified stock shall be made under this subsection no later than 30 days after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, and shall be made in a manner similar to the manner in which an election is made under subsection (b).

(B)Limitations
No election may be made under this section with respect to any qualified stock if—
(i)the qualified employee has made an election under subsection (b) with respect to such qualified stock,
(ii)any stock of the corporation which issued the qualified stock is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) at any time before the election is made, or
(iii)such corporation purchased any of its outstanding stock in the calendar year preceding the calendar year which includes the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, unless—
(I)not less than 25 percent of the total dollar amount of the stock so purchased is deferral stock, and
(II)the determination of which individuals from whom deferral stock is purchased is made on a reasonable basis.
(C)Definitions and special rules related to limitation on stock redemptions
(i)Deferral stock
For purposes of this paragraph, the term “deferral stock” means stock with respect to which an election is in effect under this subsection.

(ii)Deferral stock with respect to any individual not taken into account if individual holds deferral stock with longer deferral period
Stock purchased by a corporation from any individual shall not be treated as deferral stock for purposes of subparagraph (B)(iii) if such individual (immediately after such purchase) holds any deferral stock with respect to which an election has been in effect under this subsection for a longer period than the election with respect to the stock so purchased.

(iii)Purchase of all outstanding deferral stock
The requirements of subclauses (I) and (II) of subparagraph (B)(iii) shall be treated as met if the stock so purchased includes all of the corporation’s outstanding deferral stock.

(iv)Reporting
Any corporation which has outstanding deferral stock as of the beginning of any calendar year and which purchases any of its outstanding stock during such calendar year shall include on its return of tax for the taxable year in which, or with which, such calendar year ends the total dollar amount of its outstanding stock so purchased during such calendar year and such other information as the Secretary requires for purposes of administering this paragraph.

(5)Controlled groups
For purposes of this subsection, all persons treated as a single employer under section 414(b) shall be treated as 1 corporation.

(6)Notice requirement
Any corporation which transfers qualified stock to a qualified employee shall, at the time that (or a reasonable period before) an amount attributable to such stock would (but for this subsection) first be includible in the gross income of such employee—
(A)certify to such employee that such stock is qualified stock, and
(B)notify such employee—
(i)that the employee may be eligible to elect to defer income on such stock under this subsection, and
(ii)that, if the employee makes such an election—
(I)the amount of income recognized at the end of the deferral period will be based on the value of the stock at the time at which the rights of the employee in such stock first become transferable or not subject to substantial risk of forfeiture, notwithstanding whether the value of the stock has declined during the deferral period,
(II)the amount of such income recognized at the end of the deferral period will be subject to withholding under section 3401(i) at the rate determined under section 3402(t), and
(III)the responsibilities of the employee (as determined by the Secretary under paragraph (3)(A)(ii)) with respect to such withholding.
(7)Restricted stock units
This section (other than this subsection), including any election under subsection (b), shall not apply to restricted stock units.
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ If you want to see a possible solution, run the cell below!

In [ ]:
from hints import exercise_9_1_solution; print(exercise_9_1_solution)

### Exercise 9.2 - Codebot
In this exercise, we will write up a prompt for a **coding assistance and teaching bot that reads code and offers guiding corrections when appropriate**. Fill in the prompt element fields with content that match the description and the examples you've seen in the preceding examples of complex prompts. Once you have filled out all the prompt elements that you want to fill out, run the cell to see the concatenated prompt as well as Claude's response.

We suggest you read through the variable content (in this case, `{CODE}`) to understand what content Claude is expected to work with. Be sure to reference `{CODE}` directly in your prompt somewhere (using f-string syntax like in the other examples) so that the actual variable content can be substituted in.

In [ ]:
######################################## INPUT VARIABLES ########################################

# Input variable - the code that Claude needs to read and assist the user with correcting
CODE = """
# Function to print multiplicative inverses
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ 如果您想查看可能的解决方案，请运行下面的单元格！

In [ ]:
# 获取练习 9.2 的解决方案示例
from hints import exercise_9_2_solution; print(exercise_9_2_solution)

In [ ]:
# 提示 - 庆祝完成课程的诗歌
PROMPT = "Write an ode to a fabulous student who has just completed a course on prompt engineering, in the form of a sonnet."

# 打印Claude的响应 - 一首致敬完成提示工程课程的学生的十四行诗
print(get_completion(PROMPT))

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 对话历史（也可以在API调用中添加为前面的 `user` 和 `assistant` 消息）
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# 第二个输入变量 - 用户的问题
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## 提示元素 ########################################

##### 提示元素 1: `user` 角色
# 确保您的Messages API调用始终以消息数组中的 `user` 角色开始。
# 上面定义的get_completion()函数将自动为您完成此操作。

##### 提示元素 2: 任务背景
# 给Claude关于它应该承担的角色或您希望它通过提示承担的目标和总体任务的背景。
# 最好在提示正文的早期放置背景。
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### 提示元素 3: 语调背景
# 如果对交互很重要，告诉Claude应该使用什么语调。
# 根据任务，此元素可能不是必需的。
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### 提示元素 4: 详细任务描述和规则
# 展开您希望Claude执行的具体任务，以及Claude可能必须遵循的任何规则。
# 这也是您可以给Claude一个"出路"的地方，如果它没有答案或不知道。
# 理想情况下，向朋友展示此描述和规则，以确保它逻辑清晰且任何歧义词汇都被明确定义。
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### 提示元素 5: 示例
# 为Claude提供至少一个理想响应示例供其模仿。将其包含在<example></example> XML标签中。随意提供多个示例。
# 如果您确实提供多个示例，请给Claude关于这是什么示例的背景，并将每个示例包含在其自己的XML标签集中。
# 示例可能是知识工作中让Claude表现如期望的最有效的单一工具。
# 确保给Claude常见边缘情况的示例。如果您的提示使用便笺本，有效地给出便笺本应该如何看起来的示例。
# 通常更多示例 = 更好。
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### 提示元素 6: 要处理的输入数据
# 如果有Claude需要在提示中处理的数据，请在此处包含在相关的XML标签中。
# 随意包含多个数据片段，但确保将每个都包含在其自己的XML标签集中。
# 根据任务，此元素可能不是必需的。排序也是灵活的。
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### 提示元素 7: 即时任务描述或请求 #####
# "提醒"Claude或准确告诉Claude它预期立即要做什么来完成提示的任务。
# 这也是您放入其他变量（如用户问题）的地方。
# 向Claude重申其即时任务通常不会有害。最好在长提示的末尾执行此操作。
# 这将比将其放在开头产生更好的结果。
# 将用户查询放在提示底部附近通常也是良好的做法。
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### 提示元素 8: 预认知（逐步思考）
# 对于具有多个步骤的任务，最好告诉Claude在给出答案之前逐步思考
# 有时，您甚至可能必须说"在您给出答案之前..."只是为了确保Claude首先执行此操作。
# 不是所有提示都必需的，但如果包含，最好在长提示的末尾执行此操作，并在最终即时任务请求或描述之后。
PRECOGNITION = "Think about your answer first before you respond."

##### 提示元素 9: 输出格式
# 如果您希望Claude的响应以特定方式格式化，请明确告诉Claude该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果您包含它，将其放在提示末尾比放在开头更好。
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### 提示元素 10: 预填充Claude的响应（如果有）
# 一个用一些预填充词汇开始Claude答案以引导Claude行为或响应的空间。
# 如果您想预填充Claude的响应，您必须将其放在API调用中的 `assistant` 角色中。
# 根据任务，此元素可能不是必需的。
PREFILL = "[Joe] <response>"



######################################## 合并元素 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示
print("--------------------------- 完整提示与变量替换 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 法律文档
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# 第二个输入变量 - 用户的问题
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## 提示元素 ########################################

##### 提示元素 1: `user` 角色
# 确保您的Messages API调用始终以消息数组中的 `user` 角色开始。
# 上面定义的get_completion()函数将自动为您完成此操作。

##### 提示元素 2: 任务背景
# 给Claude关于它应该承担的角色或您希望它通过提示承担的目标和总体任务的背景。
# 最好在提示正文的早期放置背景。
TASK_CONTEXT = "You are an expert lawyer."

##### 提示元素 3: 语调背景
# 如果对交互很重要，告诉Claude应该使用什么语调。
# 根据任务，此元素可能不是必需的。
TONE_CONTEXT = ""

##### 提示元素 4: 要处理的输入数据
# 如果有Claude需要在提示中处理的数据，请在此处包含在相关的XML标签中。
# 随意包含多个数据片段，但确保将每个都包含在其自己的XML标签集中。
# 根据任务，此元素可能不是必需的。排序也是灵活的。
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### 提示元素 5: 示例
# 为Claude提供至少一个理想响应示例供其模仿。将其包含在<example></example> XML标签中。随意提供多个示例。
# 如果您确实提供多个示例，请给Claude关于这是什么示例的背景，并将每个示例包含在其自己的XML标签集中。
# 示例可能是知识工作中让Claude表现如期望的最有效的单一工具。
# 确保给Claude常见边缘情况的示例。如果您的提示使用便笺本，有效地给出便笺本应该如何看起来的示例。
# 通常更多示例 = 更好。
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### 提示元素 6: 详细任务描述和规则
# 展开您希望Claude执行的具体任务，以及Claude可能必须遵循的任何规则。
# 这也是您可以给Claude一个"出路"的地方，如果它没有答案或不知道。
# 理想情况下，向朋友展示此描述和规则，以确保它逻辑清晰且任何歧义词汇都被明确定义。
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### 提示元素 7: 即时任务描述或请求
# "提醒"Claude或准确告诉Claude它预期立即要做什么来完成提示的任务。
# 这也是您放入其他变量（如用户问题）的地方。
# 向Claude重申其即时任务通常不会有害。最好在长提示的末尾执行此操作。
# 这将比将其放在开头产生更好的结果。
# 将用户查询放在提示底部附近通常也是良好的做法。
IMMEDIATE_TASK = ""

##### 提示元素 8: 预认知（逐步思考）
# 对于具有多个步骤的任务，最好告诉Claude在给出答案之前逐步思考
# 有时，您甚至可能必须说"在您给出答案之前..."只是为了确保Claude首先执行此操作。
# 不是所有提示都必需的，但如果包含，最好在长提示的末尾执行此操作，并在最终即时任务请求或描述之后。
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### 提示元素 9: 输出格式
# 如果您希望Claude的响应以特定方式格式化，请明确告诉Claude该格式是什么。
# 根据任务，此元素可能不是必需的。
# 如果您包含它，将其放在提示末尾比放在开头更好。
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### 提示元素 10: 预填充Claude的响应（如果有）
# 一个用一些预填充词汇开始Claude答案以引导Claude行为或响应的空间。
# 如果您想预填充Claude的响应，您必须将其放在API调用中的 `assistant` 角色中。
# 根据任务，此元素可能不是必需的。
PREFILL = "<relevant_quotes>"



######################################## 合并元素 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示
print("--------------------------- 完整提示与变量替换 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))